# Lipid Metabolism WP1423

In [1]:
from pip._internal import main as pip
try:
    import csv
except ImportError:
    pip(['install', 'csv'])
    import csv
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except  ImportError:
    pip(['install', 'sparqlwrapper'])
    from SPARQLWrapper import SPARQLWrapper, JSON
    
import pandas

sparql = SPARQLWrapper("http://sparql.wikipathways.org")
lipMet = pandas.DataFrame(columns=['Enzyme', 'Metabolite Source', 'Metabolite Source Label', 'Metabolite Target',
                                   'Metabolite Target Label', 'Interaction'])

pathwayQuery = '''
SELECT DISTINCT (substr(str(?source),24 ) as ?enzymeSource) (substr(str(?metSource),24 ) as ?metaboliteSource) 
?metLabel as ?metaboliteSourceLabel (substr(str(?metTarget),24 ) as ?metaboliteTarget) ?metLabel1 as ?metaboliteTargetLabel 
?interaction   
WHERE {

   ?pathway a wp:Pathway ;
       dc:identifier <http://identifiers.org/wikipathways/WP1423> .
   ?interaction dcterms:isPartOf ?pathway ; 
       a wp:Catalysis ;
       wp:source ?source;
       wp:target ?interactionTarget .
   ?interactionTarget dcterms:isPartOf ?pathway ;
       wp:source ?metSource ; 
       wp:target ?metTarget .
   ?metSource    rdfs:label   ?metLabel .
   ?metTarget    rdfs:label   ?metLabel1 .
}
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	lipMet = lipMet.append({
		'Enzyme': result["enzymeSource"]["value"],
        'Metabolite Source': result["metaboliteSource"]["value"],
		'Metabolite Source Label': result["metaboliteSourceLabel"]["value"],
        'Metabolite Target': result["metaboliteTarget"]["value"],
		'Metabolite Target Label': result["metaboliteTargetLabel"]["value"],
        'Interaction': result["interaction"]["value"],        
	 }, ignore_index=True)

In [2]:
lipMet

,Enzyme,Metabolite Source,Metabolite Source Label,Metabolite Target,Metabolite Target Label,Interaction
0,ensembl/ENSG00000169359,hmdb/HMDB0004913,GD3,pubchem.compound/73427362,O-Acetylated GD3,http://rdf.wikipathways.org/Pathway/WP1423_r94...
1,ensembl/ENSG00000235863,hmdb/HMDB0004925,GD2,hmdb/HMDB0004926,GD1b,http://rdf.wikipathways.org/Pathway/WP1423_r94...
2,ensembl/ENSG00000101638,hmdb/HMDB0004927,GT1b,hmdb/HMDB0004928,GQ1bA,http://rdf.wikipathways.org/Pathway/WP1423_r94...
3,ensembl/ENSG00000115525,hmdb/HMDB0006750,Lactosylceramide,hmdb/HMDB0004844,GM3,http://rdf.wikipathways.org/Pathway/WP1423_r94...
4,ensembl/ENSG00000115525,hmdb/HMDB0006750,LacCer,hmdb/HMDB0004844,GM3,http://rdf.wikipathways.org/Pathway/WP1423_r94...
